In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('lab01.ok')

In [6]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('fivethirtyeight')

import pandas as pd
import zipfile
import io
import math

def css_styling():
    styles = open('../notebook_styles.css', 'r').read()
    return HTML(styles)
css_styling()

# Lab 01 - Life and death in the United States

## Speedy review of life tables

Last class, you learned some of the basics of the **period life table**. The period life table is a key concept in the study of mortality.

Let's quickly review some of the material that we covered in the brief lecture.

**Life table concepts, columns and notation**

The life tables we'll work with today describe the experience of hypothetical cohort of people who start at age 0. The mortality experience of this hypothetical cohort is given by observed mortality in a real world population (e.g. a U.S. state).  The number of people who start at age 0 in the hypothetical cohort is called the **radix**; today, we'll use life tables that have a couple of important properties:

* the radix is 100,000 people
* the width of all of the age intervals is 1

With that in mind, we'll use the following names for the columns of the life table:

* `lx` - the number of people who survive to exact age $x$ (called $l_x$ in mathematical notation)
* `dx` - the number of deaths between ages $x$ and $x + 1$ (called ${}_nd_x$ in mathematical notation)
* `qx` - the probability of dying between ages $x$ and $x+1$, given survival to age $x$ (called ${}_nq_x$ in mathematical notation)
* `mx` - the death *rate* between ages $x$ and $x+1$ in the life table population (called ${}_nm_x$ in mathematical notation)
* `Lx` - the number of person-years lived between ages $x$ and $x + 1$ by people in the life table population (called ${}_nL_x$ in mathematical notation)
* `ax` - the average number of years lived *among people who die* in the interval from $x$ to $x + 1$ (called ${}_na_x$ in mathematical notation)
* `Tx` - the total number of person-years lived above exact age $x$ (called $T_x$ in mathematical notation)
* `ex` - the (remaining) life expectancy at exact age $x$ (called $e_x$ in mathematical notation)


## Introductions

**What is your partner's name?**
<!--
BEGIN QUESTION
name: intro1
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Which U.S. State has your partner spent the most time in?**
<!--
BEGIN QUESTION
name: intro2
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Which U.S. State would your partner most like to visit in the future?**
<!--
BEGIN QUESTION
name: intro3
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

Let's start with a couple of questions that will check your understanding of life tables. (Be sure to discuss these with your partner.)

**Question** What is another name for $l_0$?
<!--
BEGIN QUESTION
name: intro4
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question** What is the relationship between `Lx`, `ax`, and `dx`?
<!--
BEGIN QUESTION
name: intro5
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question** How can life expectancy at age $x$ be calculated from other columns in the life table?
<!--
BEGIN QUESTION
name: intro5
points: 1
manual: True
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

Great -- good to review!

Moving forward, in today's class, we'll start to use what we learned last week by exploring life tables for the US states. Our goals are:

* to build our understanding of life tables
* to start to recognize some characteristic patterns of mortality in human populations
* to investigate how different human populations vary in terms of mortality
* to get some practice using the tools you're learning in Data 8

### The US Mortality Database

We'll be looking at life tables from the [United States Mortality Database](https://usa.mortality.org/), a brand-new resource that was produced by researchers here at UC Berkeley, along with collaborators at INED (the French Demographic institute) and the UN. The USMD has life tables for each US state starting several decades in the past, and going up to 2015.

You have signed up for access to the USMD as part of the first homework.

Since we will be looking at life tables for different US States, it will be helpful to have a list that has all of the two-letter  codes for the 50 states and for Washington, D.C.

In [8]:
## all of the state codes
all_states = \
       ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',\
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',\
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',\
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',\
       'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In order to make it easier to read in data from the USMD, I've written a simple function, `get_state_lt`. Look at the function and try to get a sense for how it works. (It's OK if you don't understand every detail at this point.)

In [9]:
def get_state_lt(state_code, sex_code, year = None):
    """
    Grab the life table data for a given state, sex, and time period
    
    Arguments:
      state_code - the two-letter state abbreviation, all caps (example: 'CA' or 'NY')
      sex_code - either 'f' for females, 'm' for males, or 'b' for both
      year - the year (if not used, all years are returned)
    """
    
    zipurl = os.path.join('../data/temp-us-lifetables.zip')
    
    fileurl = os.path.join('States', state_code, state_code + '_' + sex_code + 'ltper_1x1.csv')
    
    #zipurl = "https://www.dennisfeehan.org/temp-us-lifetables.zip"
    #r = requests.get(zipurl)
    #with zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    with zipfile.ZipFile(zipurl) as archive:        
        with archive.open(fileurl) as ltfile:
            lt_data_raw = pd.read_csv(ltfile)
            lt_data = Table.from_df(lt_data_raw)

    # remove the highest age group
    lt_data = lt_data.where('Age', are.not_equal_to('110+'))
    
    # convert everything but state name and sex to numeric data types
    for cur_col in lt_data.labels[2:]:
        lt_data[cur_col] = pd.to_numeric(lt_data[cur_col])
    
    if year is not None:
        lt_data = lt_data.where('Year', are.equal_to(int(year)))
    else:
        print("No year specified, so returning all years.")
        
    return(lt_data)


**Now use the `get_state_lt` function to read in the life table for New York males in 2015**

In [5]:
test_lt = get_state_lt(...)
test_lt

In [10]:
## SOLUTION
test_lt = get_state_lt('NY', 'm', 2015)
test_lt

In [ ]:
_ = lab01.grade('test_load_lt')

You can see that, in addition to the life table columns you discussed last week, a USMD life table has the following columns:

* `PopName` - the state code
* `Sex` - 'm' for males, 'f' for females, and 'b' for both
* `Year` - the period (calendar year) that the life table is based on. The most recent year is 2015.

## Visualize columns of the life table for Californian women

We'll start a deeper dive into US life tables using Californian women in 2015 as an example.

**Question: Use the `get_state_lt` function to retrieve the life table for Californan females in 2015**

In [ ]:
ca_f_lt = ...
ca_f_lt

In [11]:
## SOLUTION
ca_f_lt = get_state_lt('CA', 'f', 2015)
ca_f_lt

In [12]:
_ = lab01.grade('test_load_lt_caf')

**Question: Plot the survivorship column (lx) by age**

In [ ]:
...

In [13]:
## SOLUTION
ca_f_lt.plot('Age', 'lx')

**Question: Plot the life table number of deaths (dx) by age**

In [ ]:
...

In [14]:
## SOLUTION
ca_f_lt.plot('Age', 'dx')

**Question: Plot the life table death rate (mx) by age**

In [ ]:
...

In [15]:
## SOLUTION
ca_f_lt.plot('Age', 'mx')

Death rates like the ones you just plotted are central to the study of mortality; however, as the plot reveals, they vary tremendously across the age range.  In fact, they vary so much that it can be hard to see much helpful information on a plot like the one above.  Therefore, demographers usually look at the logarithm of the death rate, rather than the death rates themselves.

**Create a new column called logmx that has the log of the life table death rate; plot logmx by age**  
*[HINT: the function `np.log` will take the logarithm of a Series for you]*

In [ ]:
ca_f_lt = ca_f_lt.with_column('logmx', ...)
ca_f_lt

In [16]:
## SOLUTION
ca_f_lt = ca_f_lt.with_column('logmx', np.log(ca_f_lt.column('mx')))
ca_f_lt

In [17]:
_ = lab01.grade('test_logmx')

**Question: Now plot the log of the life table death rate by age**

In [ ]:
...

In [18]:
## SOLUTION
ca_f_lt.plot('Age', 'logmx')

**About what age seems to have the lowest death rate?**

[ANSWER HERE]

Last class, you discussed how to use the life table to calculate life expectancy at birth: it is the life table number of people who reach age x divided by the total person-years lived above age x.

**Question: use the CA female life table to calculate life expectancy at birth**

In [ ]:
radix = 100000 # this is the radix for all of the USMD lifetables
ca_f_e0 = ... / ...
ca_f_e0

In [19]:
## SOLUTION
radix = 100000 # this is the radix for all of the USMD lifetables
ca_f_e0 = np.sum(ca_f_lt.column('Lx')) / radix
ca_f_e0

In [20]:
_ = lab01.grade('test_manual_ex')

**Question: Now compare your calculation to the `ex` entry in the first row of the life table**

In [ ]:
... # print the first few rows of the life table to compare

In [21]:
## SOLUTION
ca_f_lt # print the first few rows of the life table to compare

We most often discuss life expectancy at birth, but life expectancy can be calculated at any age; the *ex* column of the life table has the expected number of years of life to be lived above each age.

**Question: Plot life expectancy by age**

In [ ]:
...

In [22]:
## SOLUTION
ca_f_lt.plot('Age', 'ex')

**Does anything surprise you about the plot of life expectancy by age?**

[Answer here]

## Comparing California and another state

**Question: Now plot the dx column of the life table for New Hampshire females in 2015.**  
*[NOTE: naturally, you will have to open the life table before you can plot it]*

In [ ]:
...

In [23]:
## SOLUTION
nh_f_lt = get_state_lt('NH', 'f', 2015)
nh_f_lt
nh_f_lt.plot('Age', 'dx')

**Question: ... and calculate the log death rates and plot them by age for NH females as well.**

In [ ]:
nh_f_lt = nh_f_lt.with_column('logmx', ...)
nh_f_lt.plot(...)

In [24]:
## SOLUTION
nh_f_lt = nh_f_lt.with_column('logmx', np.log(nh_f_lt.column('mx')))
nh_f_lt.plot("Age", "logmx")

**Question: What difference do you notice between the plots for NH females and the analogous plots you made earlier for CA females? What might explain this difference?**

[answer here]

[SOLUTION]
A big difference is that the curves are much less smooth for NH than they were for CA. That is likely due to the fact that NH is much smaller than California, meaning that the death rate for each age is likely to be more susceptible to random variation.

**Question: What similarities do you notice between the plot for NH females and the plot you made earlier for CA females?**

[answer here]

[SOLUTION] Taking dx as an example, both plots have the same general shape: there is an uptick at age 0, followed by a sharp decline to around age 10 or 15; then there's a smooth increase up to a single peak a little above age 80, and a decline after that.

## Comparing males and females

Now we'll start to compare some life table quantities for males and for females.

**Question: Get the life tables for California males and females in 2015**

In [ ]:
ca_lt_f = ...
ca_lt_m = ...

In [25]:
## SOLUTION
ca_lt_f = get_state_lt('CA', 'f', 2015)
ca_lt_m = get_state_lt('CA', 'm', 2015)

**Question: Now make a new table whose colums are 'Age', 'Males' (with male ex column), and 'Females' (with female ex column)**

In [ ]:
ca_lt_ex = Table().with_columns('Age', ...,
                                'Males', ...,
                                'Females', ...)
ca_lt_ex

In [26]:
## SOLUTION
ca_lt_ex = Table().with_columns('Age', ca_lt_f['Age'],
                                   'Males', ca_lt_m['ex'],
                                   'Females', ca_lt_f['ex'])
ca_lt_ex

In [27]:
_ = lab01.grade('test_manual_mf_compare')

**Question: Plot the male and female ex column by age (both males and females in one plot)**

In [ ]:
...

In [28]:
## SOLUTION
ca_lt_ex.plot('Age')

Now we'll generalize the approach above so that it is a function. Our function will be able to plot a comparison between males and females for any life table column. This will save us lots of time, and it's a good example of how writing functions can be helpful.

**Question: Complete the missing parts of the function below**

In [ ]:
def mf_compare(state, year, col):
    """
    For a given life table column, state, and year, make a plot comparing males and females.
    """

    # open the data for males and females
    lt_m = ...
    lt_f = ...
    
    # make a dataframe that has the male ad female values together
    lt_comp = Table().with_columns('Age', lt_f['Age'],
                                   'Males', ...,
                                   'Females', ...)
    
    return(lt_comp)

In [29]:
## SOLUTION
def mf_compare(state, year, col):
    """
    For a given life table column, state, and year, make a plot comparing males and females.
    """

    # open the data for males and females
    lt_m = get_state_lt(state, 'm', year)
    lt_f = get_state_lt(state, 'f', year)
    
    # make a dataframe that has the male ad female values together
    lt_comp = Table().with_columns('Age', lt_f['Age'],
                                   'Males', lt_m[col],
                                   'Females', lt_f[col])
    
    return(lt_comp)

**Question: Now use the `mf_compare` function to compare the life table age at death (the dx column) for males and females in California in 2015**

In [ ]:
ca_compare_dx = mf_compare(...)
ca_compare_dx.plot("Age")

In [30]:
## SOLUTION
ca_compare_dx = mf_compare('CA', 2015, 'dx')
ca_compare_dx.plot("Age")

In [31]:
_ = lab01.grade('test_mf_compare_dx')

**Question: Pick two other states and use the `mf_compare` function to compare the life table age at death (the dx column) for males and females in those two states**

In [ ]:
# first state
...

In [ ]:
# second state
...

In [32]:
## SOLUTION
# first state
mf_compare('TX', 2015, 'dx')

In [33]:
## SOLUTION
# second state
mf_compare('NH', 2015, 'dx')

**Question** The general shape of the male/female differences is probably reasonably similar across the three states you examined. What do you notice about male versus female ages at death across all three states?

[Answer here]

[SOLUTION]  
Females tend to die older AND there seems to be more variation in the age at death for males than for females (i.e., the age at death distribution is more spread out for males than for females)

### SUBMIT your assignment by MIDNIGHT on the day of class

Please submit your lab in **two ways** this first time:

1. **Run the cell below**, which will submit via OkPy. You can submit as many times as you want, up to midnight on the day of class.
2. As a backup, please **also submit on bCourses** by uploading the .ipynb file


In [34]:
_ = lab01.submit()

## Optional challenge problems: Exploring cross-state variation

**Question** If you have extra time, see if you can figure out   
(a) which state has the highest female life expectancy in 2015?  
(b) which state has the lowest female life expectancy in 2015?  
What might explain the variation in female life expectancy that you observe?

In [ ]:
...

In [35]:
## SOLUTION
# make a Table to keep track of results
state_e0 = Table(make_array('state', 'e0'))

# go through each state and get life expectancy at birth for females in 2015; record the results in the table
for cur_state in all_states:
    cur_lt = get_state_lt(cur_state, 'f', 2015)
    state_e0 = state_e0.with_row([cur_state, cur_lt.column('ex')[0]])

# sort the table by e0 to see which is highest and lowest
state_e0.sort('e0').show()

[SOLUTION]  
Highest female le in 2015: Hawaii (84.21)  
Lowest female le in 2015: Mississippi (77.68)

The gap is about 6.5 years

**Question** If you **still** have extra time, see if you can figure out which state has the biggest gap between male and female life expectancies.

In [ ]:
...

In [36]:
## SOLUTION
# make a Table to keep track of results
state_e0_gap = Table(make_array('state', 'e0_m', 'e0_f', 'e0_gap'))

# go through each state and get life expectancy at birth for females in 2015; record the results in the table
for cur_state in all_states:
    cur_lt_f = get_state_lt(cur_state, 'f', 2015)
    cur_e0_f = cur_lt_f.column('ex')[0]
    
    cur_lt_m = get_state_lt(cur_state, 'm', 2015)
    cur_e0_m = cur_lt_m.column('ex')[0]
    
    state_e0_gap = state_e0_gap.with_row([cur_state, cur_e0_f, cur_e0_m, cur_e0_f - cur_e0_m])

# sort the table by e0 to see which is highest and lowest
state_e0_gap.sort('e0_gap').show()

[SOLUTION]  
Highest gap is DC (6.14 years)  
Lowest gap is Utah (3.34 years) . 

### Don't forget to SUBMIT your assignment by MIDNIGHT on the day of class

If you attempted the challenge questions, great! Be sure to submit afterwards using the instructions in the cell above.

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

<!-- EXPECT 6 EXPORTED QUESTIONS -->

In [ ]:
# Save your notebook first, then run this cell to submit.
import jassign.to_pdf
jassign.to_pdf.generate_pdf('lab01.ipynb', 'lab01.pdf')
ok.submit()